<a href="https://colab.research.google.com/github/TucspDretcm/Perm-2c/blob/main/Perm_2c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><b><u>Algebra Abstracta</u></b></h1>
2022-1

Permanente 2c

<h2>Integrantes:</h2>

*   Gabriel Ivan Rodriguez Postigo
*   Jaime Mateo Gutiérrez Muñoz
*   Alexander Carpio Mamani

# RSA KEY GENERATOR

## 1.1) Euclides

In [2]:
def Euclides(a, b):
  if b == 0:
    return a
  else:
    return Euclides(b, a%b)

def Ext_Euclides(a, b):
  if b == 0:
    return (a, 1, 0)
  else:
    d, x_, y_ = Ext_Euclides(b, a%b)
    x, y = y_, x_ - int(a/b)*y_
    return (d, x, y)

def Inversa(a, n):
  if Euclides(a, n) == 1:
    mgd, x, y = Ext_Euclides(a, n)
    return x % n
  else:
    print("No existe inversa")

## 1.2) Miller-Rabin

In [3]:
import random

# Iterativo
def EXPMOD(a, x, n):
  c = a % n
  r = 1
  while (x > 0):
    if x % 2 != 0:
      r = (r * c) % n
    c = (c * c) % n
    x = int(x/2)
  return r

def ES_COMPUESTO(a, n, t, u):
  # x = a**u % n    # ERROR! : OVERFLOW con números de varias cifras.
  x = EXPMOD(a,u,n)
  if x == 1 or x == n-1:
    return False  # n es posiblemente primo
  for i in range(t):
    x = EXPMOD(x,2,n)
    if x == n-1:
      return False  # n es posiblemente primo
  return True    # n es un número compuesto

def gen_random_a(n, s):  # genera "s" numeros random unicos en el rango de 2 y n-1, si no se pueden generar "s" números entonces retorna lo que tenga.
  randoms = []
  maximo = min(s, n-3)
  while len(randoms) < maximo:
    num = random.randint(2, n-1)
    if num not in randoms:
      randoms.append(num)
  return randoms

def MILLER_RABIN(n, s):
  t = 0
  u = n - 1
  while u % 2 == 0:
    u = u//2
    t = t + 1
  #for a in gen_random_a(n, s):
  for _ in range(s):
    a = random.randint(2, n-1)
    if ES_COMPUESTO(a, n, t, u):
      return False
  return True

def RANDOMBITS(b):
  n = random.randint(0, 2**b - 1)
  m = 2**(b-1) + 1
  return n | m    # el operador "|" nos permite hacer una operación binaria de "n" y "m" en binario(bin(n) v bin(m)).

def RANDOMGEN_PRIMOS(b, s):
  n = RANDOMBITS(b)
  while MILLER_RABIN(n, s) == False:
    n = n + 2
  return n

def GEN_PRIMO_SIGUIENTE(n, s):
  n = n + 1 - (n % 2)
  while MILLER_RABIN(n, s) == False:
    n = n + 2
  return n

## 1.3) Implementar RSA KEY GENERATOR.

In [4]:
class RSA:
  def __init__(self, k, info=False):

    n = 1
    while True:
      self.e, self.d, self.n = self.RSA_KEY_GENERATOR(k, 10**n)
      m = random.randint(2,self.n-1)
      c = self.Cifrado(m)
      if m == self.Descifrado(c):
        break
      else:
        n += 1

    if info:
      print("RSA (s={:}, e={:}, d={:}, n={:}, phi={:})".format(10**n, self.e, self.d, self.n, self.phi))

  def RSA_KEY_GENERATOR(self, k, s):
    if k<8:    # generar primos con bits menores al dividirlos entre 2 no abria muchos, ocurria un loop, o un error.
      k = 8
    p = RANDOMGEN_PRIMOS(k//2, s)
    q = RANDOMGEN_PRIMOS(k//2, s)
    while p==q:
      q = RANDOMGEN_PRIMOS(k//2, s)

    n, self.phi = p*q, (p-1)*(q-1)

    e = random.randint(2,n-1)
    while Euclides(e, self.phi)!=1:
      e = random.randint(2, n-1)
      
    d = Inversa(e, self.phi)
    return e, d, n

  def Cifrado(self, m):
    return EXPMOD(m, self.e, self.n)

  def Descifrado(self, c):
    return EXPMOD(c, self.d, self.n)
  
  def Cifrado_msg(self, msg):
    cade = ""
    data = []
    for m in msg:
      me = self.Cifrado(ord(m))    # pasamos el valor en ascci de la palabra a el Cifrado RSA
      encrip = me % ord('A') + 65   # rango [65, 122] donde 65='A' y 122='z'
      cade += str("%c" % encrip)   # convertimos de decimal a word ascci
      data.append(me)
    return cade, data

  def Descifrado_msg(self, msg, data):
    cade = ""
    for i in range(len(msg)):
      med = self.Descifrado(data[i])
      cade += str("%c" % med)
    return cade

  def get_edn(self):
    return self.e, self.d, self.n

In [6]:
rsa = RSA(59, True)   # definir k = n bits

mensaje = 13
print("\nMensaje original: ", mensaje)
c = rsa.Cifrado(mensaje)
print("Cifrado: ", c)
print("Descifrado: ", rsa.Descifrado(c))

e,d,n = rsa.get_edn()

print("\ne*d mod phi =", e*d % rsa.phi)    # e*d mod n = 1 mod n
print("phi | (e*d - 1) = ", (e*d-1) % rsa.phi)   # phi | (e*d - 1)
print("mgd(e, phi)={:}\t mgd(d,phi)={:}".format(Euclides(e, rsa.phi), Euclides(d, rsa.phi)))

RSA (s=100, e=71850373597704145, d=8054978309710513, n=236435091104412581, phi=236435090130279552)

Mensaje original:  13
Cifrado:  59849540514276627
Descifrado:  13

e*d mod phi = 1
phi | (e*d - 1) =  0
mgd(e, phi)=1	 mgd(d,phi)=1


# 1. (5 points) Si m es el mensaje y c es el cifrado (ambos representados por un entero). Y además, la clave pública es P = {e, n} (en ese orden). <br> Hallar m cuando: <center> P = {65537, 999630013489} y c = 747120213790</center>


In [ ]:
e, n, c = 65537, 999630013489, 747120213790

pq = []
while len(pq) < 2:
  for i in range(2, n):
    if Euclides(i, n) != 1 and MILLER_RABIN(i, 100):
      pq.append(i)

phi = (pq[0] - 1) * (pq[1] - 1)
d = Inversa(e, phi)

m_ = EXPMOD(c, d, n)

# 2. (7 points) Si m es el mensaje y c es el cifrado (ambos representados por un entero). Y además, la clave pública es P = {e, n} (en ese orden). Hallar m cuando: 
<center>
P ={7, 35794234179725868774991807832568455403003778024228226193532908190484670252364677411513516111204504060317568667} 
c=35794234179725868774991807832568455403003778024228226193532908190484670252364677411513516052471686245831933544
</center>

<h1>Sin embargo al enviar el mismo mensaje (m) cuando e' = 11, el cifrado resulto ser</h1>

<center>
c' =357942341797258687749918078325684554030037780242282261935329081
90484670252364665786748759822531352444533388184.
</center>
<br>

In [74]:
e1 = 7
e2 = 11

c1 = 35794234179725868774991807832568455403003778024228226193532908190484670252364677411513516052471686245831933544
c2 = 35794234179725868774991807832568455403003778024228226193532908190484670252364665786748759822531352444533388184

n = 35794234179725868774991807832568455403003778024228226193532908190484670252364677411513516111204504060317568667

# x > 0:

d1, x1, y1 = Ext_Euclides(e1,e2)
d2, _, _ = Ext_Euclides(c2,n)

if d1==1 and d2==1:
  if x1 < 0:
    a = EXPMOD(Inversa(c1, n), -x1, n)
  else:
    a = EXPMOD(c1, x1, n)

  if y1 < 0:
    b = EXPMOD(Inversa(c2, n), -y1, n)
  else:
    b = EXPMOD(c2, y1, n)

  m = (a * b) % n

  print("m = ", m)
  print("\"c\" igual a \"c_ = P(m)\" ?", EXPMOD(m, e1, n)==c1)

m =  13827831465063886838020624394233523964302535713793476149786042550018831062823603673726648906292457820275857059
"c" igual a "c_ = P(m)" ? False


# 3. (8 points) Validar firmas digitales: Verificar que P(S(m)) = HASH(M) para 3 mensajes distintos, mostrando la respectiva firma σ en cada caso. Utilice la Función Hash SHA-1 para generar m a través de un texto M ( por ejemplo Hola Mundo). Utilizar b = 32 bits en el algoritmo RSA.

In [90]:
import hashlib
rsa = RSA(32)   # definimos los 32 bits

print(" M\t\t\t|\t m\t\t|\t Hash(m)\t\t\t|\t m = S(m)\t|\t m = P(S(m))")
print("-"*125)

words = ["Hola Mundo", "Juego de Monos", "pichanga fiufiu"]
for word in words:
  H = hashlib.sha1(bytes(word, encoding="utf-8")).hexdigest()
  m = int(H, 16) % rsa.n
  word_S = rsa.Descifrado(m)
  word_P = rsa.Cifrado(word_S)
  print(word + " "*(25-len(word)) + str(m) + " "*(22-len(str(m))) + H + " "*5 + str(word_S) + " "*(22-len(str(word_S))) + str(word_P))

 M			|	 m		|	 Hash(m)			|	 m = S(m)	|	 m = P(S(m))
-----------------------------------------------------------------------------------------------------------------------------
Hola Mundo               548693899             48124d6dc3b2e693a207667c32ac672414913994     982569751             548693899
Juego de Monos           1438384090            d665ddfb6fe9d2c61d287468e375b2fd92a47f15     463421013             1438384090
pichanga fiufiu          1528502847            3d81f39719f5f26c5aeba7b19f57ffe062557c1e     625484321             1528502847


# **Punto adicional para el Examen Final:** Utilizar el algoritmo RSA (b = 32) para generar y validar una firma digital. Utilizar el estándar PKCS 1 v1.5 para añadir un padding al mensaje original. Fecha límite de entrega: 08/07/22.

In [50]:
import hashlib, sys

def gen_EB(M, bits):
  H = hashlib.sha1(bytes(M, encoding="utf-8")).hexdigest()
  PS = "F" * (bits - sys.getsizeof(H) - 3)
  T = hex(RANDOMGEN_PRIMOS(bits, 100))[2:] + H
  return "0001" + PS + "00" + T


def StringToInt(EB):
  return int(EB, base=16)


def IntToString(c):
  return hex(c)[2:]


bits = 32
rsa = RSA(bits)

EB = gen_EB("Hola mundo", bits)
m = StringToInt(EB)
c = rsa.Descifrado(m)  # m^d mod n
OB = IntToString(c)

print("\"m\" original: ", m % rsa.n)    # "m" original aplicando modulo para normalizar tanto la original como la "m" recuperada
print("Firma digital:", OB)
print("\"m\" recuperada: ", rsa.Cifrado(c))

"m" original:  1481651286
Firma digital: 76cda0f1
"m" recuperada:  1481651286
